# Script for running smartpca

## Before you begin

In [ ]:
#Here it is assumed that all programs are installed in directory 'bin'.  
#If they are elsewhere, the paths to individual programs have to adjusted.
cd star_protocols_saimaa

work=$(pwd)

apps=$work/bin

bcftools=$apps/bcftools
vcftools=$apps/vcftools
convertf=$apps/convertf
smartpca=$apps/smartpca


## PCA

In [ ]:
#Test with first 20 contigs

data=processed_data/2.masking-filtering/saimaa_20_phased_posmask
processed=processed_data/3.smartpca/saimaa_20_phased_posmask
results=results/3.smartpca/saimaa_20_phased_posmask

$bcftools view $data.vcf.gz | \
 perl -pe '($c)=/([0-9]+)F\|quiver\|pilon/;++$c;$c=~s/^0+//;s/([0-9]+)F\|quiver\|pilon/$c/' | \
 $vcftools --gzvcf - --plink --out $processed

cat > processed_data/3.smartpca/CONVERT << EOF
genotypename:    $processed.ped
snpname:         $processed.map 
indivname:       $processed.ped 
outputformat:    EIGENSTRAT
genotypeoutname: $processed.geno
snpoutname:      $processed.snp
indivoutname:    $processed.ind
familynames:     NO
EOF

$convertf -p processed_data/3.smartpca/CONVERT


cat > processed_data/3.smartpca/SMARTPCA << EOF
genotypename:    $processed.geno
snpname:         $processed.snp
indivname:       $processed.ind 
evecoutname:     $results.evec
evaloutname:     $results.eval
numoutevec:      10
numthreads:      4
EOF

$smartpca -p processed_data/3.smartpca/SMARTPCA > $results.out


**Note:** Eigensoft does not accept long contig names but requires the chromosomes to be numbers starting from one. The Perl command above deletes a fixed string from the contig name and, as the original numbers are from zero, adds one. There would be many other ways to achieve the same result.

**Note:** Eigensoft has an upper limit of a 100 chromosomes. If you are working with a draft assembly with > 100 contigs, you need to change parameter ``MAXCH`` in ``EIG/include/mcio.h`` before compiling the software to fit the number of contigs/chromosomes in your dataset.